In [1]:
import gc
import glob
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, StratifiedKFold
from tqdm import tqdm

from keras import optimizers
from keras.callbacks import *
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import *
from keras.models import Model, load_model, save_model
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.applications.resnet50 import ResNet50, preprocess_input

%matplotlib inline

Using TensorFlow backend.


# 【問題1】コードレビュー
転移学習を使用してセグメンテーションの精度を改善したコードを提示するので、レビューを行ってください。

《視点例》

前回使用した実装とはどのように違うのか
転移学習をどのように行っているか

In [ ]:
# Basic decoder block with Conv, BN and PReLU activation.
def decoder_block_simple(
        layer_name, block_name,
        num_filters=32,
        conv_dim=(3, 3)):

    x_dec = Conv2D(
        num_filters, conv_dim,
        padding='same',
        name='{}_conv'.format(block_name))(layer_name)
    x_dec = BatchNormalization(
        name='{}_bn'.format(block_name))(x_dec)
    x_dec = PReLU(
        name='{}_activation'.format(block_name))(x_dec)

    return x_dec

# Decoder block with bottleneck architecture, where middle conv layer
# is half the size of first and last, in order to compress representation.
# This type of architecture is supposed to retain most useful information.
# ボトルネックアーキテクチャを備えたデコーダーブロック。表現を圧縮するために、
# 中間コンバージョンレイヤーが最初と最後の半分のサイズです。 
# このタイプのアーキテクチャは、最も有用な情報を保持することになっています。
def decoder_block_bottleneck(
        layer_name, block_name,
        num_filters=32,
        conv_dim=(3, 3),
        dropout_frac=0.2):

    x_dec = Conv2D(
        num_filters, conv_dim,
        padding='same',
        name='{}_conv1'.format(block_name))(layer_name)
    x_dec = BatchNormalization(
        name='{}_bn1'.format(block_name))(x_dec)
    x_dec = PReLU(
        name='{}_activation1'.format(block_name))(x_dec)
    x_dec = Dropout(dropout_frac)(x_dec)

    x_dec2 = Conv2D(
        num_filters // 2, conv_dim,
        padding='same',
        name='{}_conv2'.format(block_name))(x_dec)
    x_dec2 = BatchNormalization(
        name='{}_bn2'.format(block_name))(x_dec2)
    x_dec2 = PReLU(
        name='{}_activation2'.format(block_name))(x_dec2)
    x_dec2 = Dropout(dropout_frac)(x_dec2)

    x_dec2 = Conv2D(
        num_filters, conv_dim,
        padding='same',
        name='{}_conv3'.format(block_name))(x_dec2)
    x_dec2 = BatchNormalization(
        name='{}_bn3'.format(block_name))(x_dec2)
    x_dec2 = PReLU(
        name='{}_activation3'.format(block_name))(x_dec2)
    x_dec2 = Dropout(dropout_frac)(x_dec2)

    x_dec2 = Add()([x_dec, x_dec2])

    return x_dec2

In [ ]:
# Model is parametrized in a way to enable easy change of decoder_block type,
# as this is an argument that can be given a function, like decoder_block_simple.
# モデルは、decoder_blockタイプを簡単に変更できるようにパラメーター化されます。
# これは、decoder_block_simpleなどの関数を指定できる引数であるためです。
def unet_resnet(input_size, decoder_block,
                weights='imagenet',
                loss_func='binary_crossentropy',
                metrics_list=[my_iou_metric],
                use_lovash=False):

    # Base model - encoder
    base_model = ResNet50(
        input_shape=input_size, 
        include_top=False,
        weights=weights)
    
    # Layers for feature extraction in the encoder part
    # エンコーダー部分での特徴抽出のためのレイヤー
    encoder1 = base_model.get_layer('activation_1').output
    encoder2 = base_model.get_layer('activation_10').output
    encoder3 = base_model.get_layer('activation_22').output
    encoder4 = base_model.get_layer('activation_40').output
    encoder5 = base_model.get_layer('activation_49').output

    # Center block
    center = decoder_block(
        encoder5, 'center', num_filters=512)
    concat5 = concatenate([center, encoder5], axis=-1)

    # Decoder part.
    # Every decoder block processed concatenated output from encoder and decoder part.
    # This creates skip connections.
    # Afterwards, decoder output is upsampled to dimensions equal to encoder output part.
    #デコーダー部分。 
    #すべてのデコーダーブロックは、エンコーダーとデコーダー部分からの連結出力を処理しました。 
    #これはスキップ接続を作成します。 
    #その後、デコーダー出力は、エンコーダー出力部分に等しい次元にアップサンプリングされます。
    decoder4 = decoder_block(
        concat5, 'decoder4', num_filters=256)
    concat4 = concatenate([UpSampling2D()(decoder4), encoder4], axis=-1)

    decoder3 = decoder_block(
        concat4, 'decoder3', num_filters=128)
    concat3 = concatenate([UpSampling2D()(decoder3), encoder3], axis=-1)

    decoder2 = decoder_block(
        concat3, 'decoder2', num_filters=64)
    concat2 = concatenate([UpSampling2D()(decoder2), encoder2], axis=-1)

    decoder1 = decoder_block(
        concat2, 'decoder1', num_filters=64)
    concat1 = concatenate([UpSampling2D()(decoder1), encoder1], axis=-1)

    # Final upsampling and decoder block for segmentation.
    output = UpSampling2D()(concat1)
    output = decoder_block(
        output, 'decoder_output', num_filters=32)
    output = Conv2D(
        1, (1, 1), activation=None, name='prediction')(output)
    if not use_lovash:
        output = Activation('sigmoid')(output)
        
    model = Model(base_model.input, output)
    model.compile(loss=loss_func, optimizer='adam', metrics=metrics_list)

    return model

## コードリーディング
・5行目～9行目：decoder_blockタイプを容易に変更できるようunet_resnet関数の引数に設定されている。  
・5行目～9行目：unet_resnetの引数で重みがimagenetで学習済みのものを使用するよう設定されている。(転移学習）  
・11行目～15行目：前回（U-net ）では各層ごとにに書かれていたが、resnetでは転移学習を行うためにencoder部分を各層ごとにインスタンス化している。  
・25～28行目：U字の最下層部分,center部分畳み込み層、バッチノーマライゼーション（正規化）、PRELU活性化関数を通し、centerレイヤーとencoder5レイヤーを連結している。  
・38行目～52行目：デコーダー部分。各層でdecoder_block（CNN）処理し、upsamplingしencoder層の出力と連結している（スキップ処理を行っている）。
それを各層で行っている。  
・56～62行目：出力層ではスキップ処理してものを受け取り、upsamplingし、CNN処理して、畳み込み処理している。
loss関数がuse_lovashなら恒等関数、loss関数がuse_lovashを使わないなら活性化関数をsigmoidとし、二値分類なのでクラス数を1として出力するようにしている。  
・64～67行目：functionalAPIで入力をbase_modelとし、Modelをインスタンス化し、コンパイルし、modelを返している。

 # 【問題2】コードの書き換え
エンコーダーにResNetが使用されていたコードをVGGに変更してください。

In [ ]:
"""unet_resnetをunet_vggに書き換え"""  
def unet_vgg(input_size, decoder_block,  
               weights='imagenet',  
                loss_func='binary_crossentropy',  
                metrics_list=[my_iou_metric],  
                use_lovash=False):  

    # Base model - encoder  
    '''ResNet50をVGG16に書き換え'''
    base_model = VGG16(  
        input_shape=input_size,   
        include_top=False,  
        weights=weights)
    
    # Layers for feature extraction in the encoder part
    # エンコーダー部分での特徴抽出のためのレイヤー
    """layernameを書き換え"""
    encoder1 = base_model.get_layer('block1_conv2').output
    encoder2 = base_model.get_layer('block2_conv2').output
    encoder3 = base_model.get_layer('block3_conv3').output
    encoder4 = base_model.get_layer('block4_conv3').output
    encoder5 = base_model.get_layer('block5_conv3').output

# 【問題3】学習・推定
ResNetとVGG双方のコードで学習・推定を行い、結果を比較してください。

# すいません、現状ここまでです。